# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

In [2]:
tfe = tf.contrib.eager

from gauge_model import (
    GaugeModel, check_else_make_dir, save_params_to_pkl_file
)

import utils.gauge_model_helpers as helpers

In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [4]:
params = {
###################### Lattice parameters ############################
    'time_size': 12,
    'space_size': 12,
    'link_type': 'U1',
    'dim': 2,
    'num_samples': 6,
    'rand': False,
###################### Leapfrog parameters ###########################
    'num_steps': 5,
    'eps': 0.2,
    'loss_scale': 1.,
    'loss_eps': 1e-4,
###################### Learning rate parameters ######################
    'learning_rate_init': 1e-3,
    'learning_rate_decay_steps': 500,
    'learning_rate_decay_rate': 0.96,
###################### Annealing rate parameters #####################
    'annealing': True,
    'annealing_steps': 200,
    'annealing_factor': 0.97,
    'beta': 2.,
    'beta_init': 2.,
    'beta_final': 8.,
###################### Training parameters ###########################
    'train_steps': 10000,
    'save_steps': 1000,
    'logging_steps': 50,
    'training_samples_steps': 500,
    'training_samples_length': 100,
###################### Model parameters ##############################
    'conv_net': True,
    'hmc': False,
    'eps_trainable': True,
    'metric': 'l2',
    'aux': True,
    'clip_grads': False,
    'clip_value': 10.,
}

## Create model

In [5]:
os.environ["KMP_BLOCKTIME"] = str(0)
os.environ["KMP_SETTINGS"] = str(1)
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"
#os.environ["OMP_NUM_THREADS"] = str(2)

In [6]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [ ]:
model = GaugeModel(params=params, 
                   config=config,
                   sess=None,
                   log_dir=None,
                   restore=False)

save_params_to_pkl_file(params, model.info_dir)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_14/
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_14/samples_history.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_14/samples_history/training.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_14/train_samples.
time_size: 12
space_size: 12
link_type: U1
dim: 2
num_samples: 6
rand: False
num_steps: 5
eps: 0.2
loss_scale: 1.0
loss_eps: 0.0001
learning_rate_init: 0.001
learning_rate_decay_steps: 500
learning_rate_decay_rate: 0.96
annealing: True
annealing_steps: 200
annealing_factor: 0.97
beta: 2.0
beta_init: 2.0
beta_final: 8.0
train_steps: 10000
save_steps: 1000
logging_steps: 50
training_samples_steps: 500
training_samples_length: 100
conv_net: True
hmc: False
eps_trainable: True
metric: l2
aux: True
clip_grads: False
clip_value: 10.0
--------------------------------------------------------------------------------

Building graph... (started at: We

In [ ]:
model.train(model.train_steps, kill_sess=False)

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------
     0/10000      -373.3        0.1967        0.07933         0.2            2           0.001    
     1/10000       2749        0.000268         0.5          0.199           2           0.001    
     2/10000       7607        0.000124       0.1667        0.1981           2           0.001    
     3/10000       1e+04       0.0001336         0          0.1974           2           0.001    
     4/10000       1e+04       0.0001418     2.365e-29      0.1968           2           0.001    
     5/10000       1e+04       0.0001881         0          0.1964           2           0.001    
     6/10000       1e+04       0.0001678         0          0.1959           2           0.001    
     

    80/10000       1e+04       0.0001705         0          0.1922           2           0.001    
    81/10000       1e+04       0.0001603         0          0.1922           2           0.001    
    82/10000       1e+04       0.0001667         0          0.1922           2           0.001    
    83/10000       1e+04       0.0001691         0          0.1922           2           0.001    
    84/10000       1e+04       0.0001293         0          0.1922           2           0.001    
    85/10000       1e+04       0.0001628         0          0.1922           2           0.001    
    86/10000       1e+04       0.0001577         0          0.1922           2           0.001    
    87/10000       1e+04       0.0001244         0          0.1922           2           0.001    
    88/10000       1e+04        0.00013          0          0.1922           2           0.001    
    89/10000       1e+04       0.0001334         0          0.1922           2           0.001    
    90/100

   163/10000       1e+04       0.0001537         0          0.1922           2           0.001    
   164/10000       1e+04       0.000158          0          0.1922           2           0.001    
   165/10000       1e+04       0.0001566         0          0.1922           2           0.001    
   166/10000       1e+04       0.0001878         0          0.1922           2           0.001    
   167/10000       1e+04       0.0001785         0          0.1922           2           0.001    
   168/10000       1e+04       0.0001665         0          0.1922           2           0.001    
   169/10000       1e+04       0.0001822         0          0.1922           2           0.001    
   170/10000       1e+04       0.0001677         0          0.1922           2           0.001    
   171/10000       1e+04       0.0001643         0          0.1922           2           0.001    
   172/10000       1e+04       0.000165          0          0.1922           2           0.001    
   173/100

   246/10000       1e+04       0.0001338         0          0.1922         2.062         0.001    
   247/10000       1e+04       0.0001131         0          0.1922         2.062         0.001    
   248/10000       1e+04       0.0001096         0          0.1922         2.062         0.001    
   249/10000       1e+04       0.0001148         0          0.1922         2.062         0.001    
   250/10000       1e+04       0.0001179         0          0.1922         2.062         0.001    
   251/10000       1e+04       0.0001361         0          0.1922         2.062         0.001    
   252/10000       1e+04       0.0001219         0          0.1922         2.062         0.001    
   253/10000       1e+04       0.0001538         0          0.1922         2.062         0.001    
   254/10000       1e+04       0.0001508         0          0.1922         2.062         0.001    
   255/10000       1e+04       0.0001381         0          0.1922         2.062         0.001    
   256/100

   329/10000       1e+04       0.0001764         0          0.1922         2.062         0.001    
   330/10000       1e+04       0.0001836         0          0.1922         2.062         0.001    
   331/10000       1e+04       0.0001827         0          0.1922         2.062         0.001    
   332/10000       1e+04       0.0001803         0          0.1922         2.062         0.001    
   333/10000       1e+04       0.0001681         0          0.1922         2.062         0.001    
   334/10000       1e+04       0.0001732         0          0.1922         2.062         0.001    
   335/10000       1e+04       0.0001374         0          0.1922         2.062         0.001    
   336/10000       1e+04       0.0001184         0          0.1922         2.062         0.001    
   337/10000       1e+04       0.0001397         0          0.1922         2.062         0.001    
   338/10000       1e+04       0.0001352         0          0.1922         2.062         0.001    
   339/100

   412/10000       1e+04       0.0001705         0          0.1922         2.126         0.001    
   413/10000       1e+04       0.0001501         0          0.1922         2.126         0.001    
   414/10000       1e+04       0.0001778         0          0.1922         2.126         0.001    
   415/10000       1e+04       0.0001748         0          0.1922         2.126         0.001    
   416/10000       1e+04       0.0001651         0          0.1922         2.126         0.001    
   417/10000       1e+04       0.0001405         0          0.1922         2.126         0.001    
   418/10000       1e+04       0.0001325         0          0.1922         2.126         0.001    
   419/10000       1e+04       0.000194          0          0.1922         2.126         0.001    
   420/10000       1e+04       0.000155          0          0.1922         2.126         0.001    
   421/10000       1e+04       0.0001712         0          0.1922         2.126         0.001    
   422/100

   495/10000       1e+04       0.0001529         0          0.1922         2.126         0.001    
   496/10000       1e+04       0.0001641         0          0.1922         2.126         0.001    
   497/10000       1e+04       0.0001625         0          0.1922         2.126         0.001    
   498/10000       1e+04       0.0001722         0          0.1922         2.126         0.001    
   499/10000       1e+04       0.0001492         0          0.1922         2.126         0.001    
--------------------------------------------------------------------------------

Evaluating sampler for 100 steps at beta = 8.0.
Running (trained) L2HMC sampler for 100 steps...


In [ ]:
run_steps = [100, 200, 500, 1000, 5000]#, 10000]

In [ ]:
for steps in run_steps:
    _ = model.run(steps)

In [39]:
import pickle

with open(model.files['parameters_pkl_file'], 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)